In [1]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.visual.training_curves import Plotter

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# initialize embeddings. This takes time to load.
embedding_types: List[TokenEmbeddings] = [
    # GloVe embeddings for arabic
    WordEmbeddings('ar'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

2019-04-13 21:04:30,951 this function is deprecated, use smart_open.open instead


In [3]:

# 1. get the corpus
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train_full.txt', 
  dev_file='dev.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


# initialize sequence tagger
from flair.models import SequenceTagger

# have a relatively small hidden_size
tagger: SequenceTagger = SequenceTagger(hidden_size=96,
                                        dropout = 0.2,
                                        rnn_layers = 2,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)# initialize trainer
# 6. Initialize trainer
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from flair.optim import AdamW
# optimizer = optimizer(momentum=0.9, dampening=0.9)
trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=AdamW)


2019-04-13 21:04:33,773 Reading data from /home/jupyter/data_ner
2019-04-13 21:04:33,774 Train: /home/jupyter/data_ner/train_full.txt
2019-04-13 21:04:33,775 Dev: /home/jupyter/data_ner/dev.txt
2019-04-13 21:04:33,777 Test: None


In [4]:
import os
# 7. start training
result_folder = '/home/jupyter/result_96_2_l01_w01_d02_adamw/'
trainer.train(result_folder,
              EvaluationMetric.MICRO_F1_SCORE,
              learning_rate=0.1,
              mini_batch_size=32,
              weight_decay = 0.1,
              max_epochs=200,
              checkpoint=False)


2019-04-13 21:04:37,924 ----------------------------------------------------------------------------------------------------
2019-04-13 21:04:37,925 Evaluation method: MICRO_F1_SCORE
2019-04-13 21:04:37,928 ----------------------------------------------------------------------------------------------------
2019-04-13 21:04:38,303 epoch 1 - iter 0/38 - loss 95.48638916
2019-04-13 21:04:39,278 epoch 1 - iter 3/38 - loss 126.89060974
2019-04-13 21:04:40,222 epoch 1 - iter 6/38 - loss 103.12296241
2019-04-13 21:04:41,194 epoch 1 - iter 9/38 - loss 77.66500349
2019-04-13 21:04:41,966 epoch 1 - iter 12/38 - loss 64.25871548
2019-04-13 21:04:42,860 epoch 1 - iter 15/38 - loss 55.49565798
2019-04-13 21:04:44,069 epoch 1 - iter 18/38 - loss 50.65910661
2019-04-13 21:04:44,896 epoch 1 - iter 21/38 - loss 46.29191763
2019-04-13 21:04:45,909 epoch 1 - iter 24/38 - loss 42.81483139
2019-04-13 21:04:46,727 epoch 1 - iter 27/38 - loss 39.89631377
2019-04-13 21:04:47,915 epoch 1 - iter 30/38 - loss 37

{'test_score': 0.786,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0033,
  0.0661,
  0.0165,
  0.0402,
  0.0,
  0.049,
  0.0533,
  0.2329,
  0.2045,
  0.3782,
  0.3408,
  0.2868,
  0.293,
  0.278,
  0.3645,
  0.2967,
  0.3694,
  0.3886,
  0.2518,
  0.4408,
  0.2629,
  0.2745,
  0.4254,
  0.3793,
  0.2681,
  0.338,
  0.3065,
  0.2857,
  0.4662,
  0.4577,
  0.4204,
  0.3146,
  0.4025,
  0.4423,
  0.4835,
  0.538,
  0.5289,
  0.5408,
  0.6009,
  0.5604,
  0.5638,
  0.5539,
  0.5808,
  0.5744,
  0.5883,
  0.5744,
  0.6245,
  0.5566,
  0.5905,
  0.6082,
  0.6099,
  0.6181,
  0.6562,
  0.6049,
  0.6166,
  0.6258,
  0.6285,
  0.653,
  0.6319,
  0.6229,
  0.6497,
  0.672,
  0.691,
  0.6794,
  0.6598,
  0.6686,
  0.6822,
  0.692,
  0.7046,
  0.7052,
  0.7126,
  0.7134,
  0.7167,
  0.7271,
  0.7111,
  0.7126,
  0.7099,
  0.7241,
  0.7301,
  0.7404,
  0.7464,
  0.7398,
  0.7527,
  0.7632,
  0.7519,
  0.7633,
  0.7686,
  0.7558,
  0.7679,
  0.7614,
  0.7747,
  0.772,
 

In [9]:
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves(os.path.join(result_folder, 'loss.tsv'))
plotter.plot_weights(os.path.join(result_folder, 'weights.txt'))